In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import io

In [2]:
# Load and join the dataframes 
wiki_art_imgs = pd.read_csv("wikiart_df.csv")
aic_art_imgs = pd.read_csv("aic_images.csv")


In [3]:
aic_art_imgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6739 entries, 0 to 6738
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   artwork_type_title  6739 non-null   object
 1   style_title         3550 non-null   object
 2   id                  6739 non-null   int64 
 3   image_id            6679 non-null   object
 4   title               6739 non-null   object
 5   search_term         6739 non-null   object
 6   file_name           6610 non-null   object
dtypes: int64(1), object(6)
memory usage: 368.7+ KB


Check for duplicates
check to make sure files exist
check for corrupted or blank images

In [ ]:
Calculate basic statistics (# of classes, images per class, image sizes)

Visualizations
- Create sample visualizations of images across different styles
- [ ] Generate distribution plots of image properties
- [ ] Analyze color distributions
- [ ] Create dimensionality reduction visualizations (t-SNE, UMAP)

Resize Images

Normalize Pixel Values

Apply Augmentations